In [2]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
    model_name="llama-3.1-70b-versatile"
)

response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped onto the lunar surface on July 20, 1969, as part of the Apollo 11 mission.


In [3]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://careers.ibm.com/job/19548519/industrial-distribution-services-it-solution-consultant-remote/?codes=WEB_SEARCH_INDIA")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.




















Industrial & Distribution Services, IT Solution Consultant - JP | IBM
















































 



Skip to content



Toggle NavigationIBM CareersEnglish简体中文FrançaisFrançais Canadien日本語PortuguêsEspañolDeutschIBM Careers IBM Careers      English    简体中文    Français    Français Canadien    日本語    Português    Español    Deutsch      
 





Job Details2024-01-04T09:25:13+00:00 

675658BR


Industrial & Distribution Services, IT Solution Consultant     Tokyo, JP   Consulting Professional   Apply now Industrial & Distribution Services, IT Solution Consultant     Tokyo, JP   Consulting Professional   Apply now IntroductionA career in IBM Consulting is rooted by long-term relationships and close collaboration with clients across the globe.You’ll work with visionaries across multiple industries to improve the hybrid cloud and AI journey for the most innovative and valuable companies in the world. Your ability to accelerate impact and make meaningful c

In [4]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
print(res.content)

[
  {
    "role": "Industrial & Distribution Services, IT Solution Consultant",
    "experience": [
      "Consulting experience in manufacturing and distribution areas in consultation firms",
      "Project experience in manufacturing and distribution areas in IT vendors or SIer",
      "Experience in information systems, IT planning",
      "Manufacturing and distribution industry solutions planning, sales experience",
      "Manufacturing and distribution industry operations, and systems development and design experience (PLM、GOM、MES、SCM、CRM、SAP、FSI、Oracle, etc.)"
    ],
    "skills": [
      "Planning a digital strategy based on a management strategy",
      "Design new solutions and drafting proposals using the latest technologies",
      "Promotion and management of systems development projects to enable IT transformation",
      "Build approaches and relationships with CxO and LOBs",
      "Renewal of existing solutions, new services and new feature proposals",
      "Promotion 

In [5]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

[{'role': 'Industrial & Distribution Services, IT Solution Consultant',
  'experience': ['Consulting experience in manufacturing and distribution areas in consultation firms',
   'Project experience in manufacturing and distribution areas in IT vendors or SIer',
   'Experience in information systems, IT planning',
   'Manufacturing and distribution industry solutions planning, sales experience',
   'Manufacturing and distribution industry operations, and systems development and design experience (PLM、GOM、MES、SCM、CRM、SAP、FSI、Oracle, etc.)'],
  'skills': ['Planning a digital strategy based on a management strategy',
   'Design new solutions and drafting proposals using the latest technologies',
   'Promotion and management of systems development projects to enable IT transformation',
   'Build approaches and relationships with CxO and LOBs',
   'Renewal of existing solutions, new services and new feature proposals',
   'Promotion of sales to achieve the Solution Business Target (Contract

In [6]:
type(json_res)

list

In [7]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [8]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

C:\Users\user\.cache\chroma\onnx_models\all-MiniLM-L6-v2\onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:41<00:00, 2.02MiB/s]


In [17]:
job=json_res


In [22]:
links = collection.query(query_texts=job[0]['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/android-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/ios-ar-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/java-portfolio'}],
 [{'links': 'https://example.com/android-portfolio'},
  {'links': 'https://example.com/java-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/typescript-frontend-portfolio'},
  {'links': 'https://example.com/full-stack-js-portfolio'}],
 [{'links': 'https://example.com/angular-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/full-stack-js-portfolio'}],
 [{'lin

In [31]:
name="Saptarshi Parui Parui"
year=2025
college="MCKV"
department="AIML"


In [32]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
    
        ### INSTRUCTION:
        You are {name}, a fresher graduating in {year}, from {college} in {department}. You are applying for the job posting mentioned above.
        Your goal is to write a cold email to the HR department, showcasing your skills, academic background, and enthusiasm for the role.
        Highlight your relevant projects and achievements to demonstrate your suitability for the position.
        Also, include the most relevant ones from the following portfolio links and experiences to showcase your capabilities: {link_list}.
        Remember to focus on how your experiences align with the specific job requirements.
        Do not provide a preamble.
    
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job),"name": name, "link_list": links,"year":year,"college":college,"department":department})
print(res.content)

Subject: Application for Industrial & Distribution Services, IT Solution Consultant Role

Dear Hiring Manager,

I am excited to apply for the Industrial & Distribution Services, IT Solution Consultant position at IBM Consulting. As a soon-to-be graduate in Artificial Intelligence and Machine Learning (AIML) from MCKV in 2025, I am confident that my academic background, skills, and enthusiasm for the role make me an ideal candidate.

Throughout my academic journey, I have developed a strong foundation in AI, machine learning, and data analysis. I have worked on various projects that demonstrate my ability to design and implement innovative solutions. One of my notable projects is a predictive maintenance system for manufacturing industries, where I utilized machine learning algorithms to predict equipment failures and optimize maintenance schedules. This project showcases my ability to apply AI and machine learning concepts to real-world problems, aligning with the job requirement of "P